In [1]:
import re

import pandas     as pd
import numpy      as np
import seaborn    as sns
import umap.umap_ as umap

from sklearn.manifold     import TSNE
from sqlalchemy           import create_engine
from sklearn.neighbors    import NearestNeighbors
from sklearn              import decomposition as dd
from matplotlib           import pyplot        as plt
from sklearn              import cluster       as c
from sklearn              import metrics       as m
from plotly               import express       as px
from sklearn              import preprocessing as pp
from sklearn              import ensemble      as en
from sklearn              import mixture       as mx
from scipy.cluster        import hierarchy     as hc

## Loading Data

In [2]:
df_raw = pd.read_csv('C:/Users/Pedro/repos/pa005_insiders_clustering/insiders_cluster/data/raw/data.csv', encoding='unicode_escape')

# 1.0 Data Description

In [3]:
df1 = df_raw.copy()

## 1.1 Rename Columns

In [4]:
cols_new = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country']

df1.columns = cols_new

## 1.2 Data Dimensions

In [5]:
print('The number of rows is: {}'.format( df1.shape[0] ))
print('The number of columns is: {}'.format( df1.shape[1] ))

The number of rows is: 541909
The number of columns is: 8


## 1.3 Data Types

In [6]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4 Check NA

In [7]:
df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5 Replace NA

In [8]:
# Spliting NA's into two differents dataframes

df_missing = df1.loc[ df1['customer_id'].isna(), : ]
df_not_missing = df1.loc[~df1['customer_id'].isna(), :]

In [9]:
# Creating Reference
df_backup = pd.DataFrame( df_missing['invoice_no'].drop_duplicates() )
df_backup['customer_id'] = np.arange( 19000, 19000+len( df_backup ), 1 )

# Merge original with reference dataframe
df1 = pd.merge( df1, df_backup, on='invoice_no', how='left' )

# coalesce
df1['customer_id'] = df1['customer_id_x'].combine_first( df1['customer_id_y'] )

# Dropping extra columns
df1 = df1.drop(columns=['customer_id_x', 'customer_id_y'], axis=1)

## 1.6 Change Types

In [10]:
# Invoice Date
df1['invoice_date'] = pd.to_datetime( df1['invoice_date'] )

# Customer ID
df1['customer_id'] = df1['customer_id'].astype(np.int64)

# 2.0 Filtragem de Variaveis

In [11]:
df2 = df1.copy()

In [12]:
# ========= Numerical Attributes ==========

# Pegando preços unitários maiores que 0.04
df2 = df2.loc[ df2['unit_price'] >= 0.04, : ]


# ========= Categorical Attributes ==========

# Removing Stock Code letters
df2 = df2[~df2['stock_code'].isin( ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'] ) ]

# Description
df2 = df2.drop( columns=['description'], axis=1 )

# Map
df2 = df2[~df2['country'].isin( [ 'european Community', 'Unpecified' ] ) ]


# Bad Users
df2 = df2[~df2['customer_id'].isin( [16446] )]


# Dividindo datasets em compras e retornos
df2_purchases = df2.loc[df2['quantity'] >= 0, :]
df2_returns = df2.loc[df2['quantity'] < 0, : ]

# 3.0. Feature Engineering

In [13]:
df3 = df2.copy()

## 3.1. Feature Creation

In [14]:
# Data reference
df_ref = df3.drop( ['invoice_no', 'stock_code', 'quantity', 'invoice_date', 'unit_price', 'country'], axis=1 ).drop_duplicates(ignore_index=True)

### 3.1.1 Gross Revenue

In [15]:
# Gross Revenue (Faturamento) quantity * price
df2_purchases.loc[:, 'gross_revenue'] = df2_purchases.loc[:, 'quantity'] * df2_purchases.loc[:, 'unit_price']

# Monetary
df_monetary = df2_purchases.loc[:, ['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge( df_ref, df_monetary, on='customer_id', how='left' )
df_ref.isna().sum()

C:\Users\Pedro\AppData\Local\Temp\ipykernel_12760\4106628865.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_purchases.loc[:, 'gross_revenue'] = df2_purchases.loc[:, 'quantity'] * df2_purchases.loc[:, 'unit_price']


customer_id       0
gross_revenue    91
dtype: int64

### 3.1.2 Recency - Days from last purchase

In [16]:
# Recency - Last Day Purchase
df_recency = df2_purchases.loc[:, ['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = ( df3['invoice_date'].max() - df_recency['invoice_date'] ).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
df_ref = pd.merge( df_ref, df_recency, on='customer_id', how='left' )
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
dtype: int64

### 3.1.3 Quantity of products purchased (variety)

In [17]:
df_stock = df2_purchases.loc[:, ['customer_id', 'stock_code']].groupby('customer_id').count().reset_index().rename(columns={'stock_code':'qnt_products'})
df_ref = pd.merge( df_ref, df_stock, on='customer_id', how='left' )
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qnt_products     91
dtype: int64

### 3.1.4 Frequency Purchase

In [18]:
df_aux = ( df2_purchases[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                             .groupby('customer_id')
                                                             .agg( max_ = ( 'invoice_date', 'max' ),
                                                                   min_ = ( 'invoice_date', 'min' ),
                                                                   days_ = ( 'invoice_date', lambda x: ( ( x.max() - x.min() ).days ) + 1 ),
                                                                   buy_ = ( 'invoice_no', 'count') ) ).reset_index()

df_aux['frequency'] = df_aux[['buy_', 'days_']].apply( lambda x: x['buy_'] / x['days_'] if x['days_'] != 0 else 0, axis=1 )


df_ref = pd.merge( df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left' )

df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qnt_products     91
frequency        91
dtype: int64

### 3.1.5 Number of Returns

In [19]:
df_returns = df2_returns[['customer_id', 'quantity']].groupby( 'customer_id' ).sum().reset_index().rename(columns={'quantity': 'qnt_returns'})
df_returns['qnt_returns'] = df_returns['qnt_returns'] * -1

df_ref = pd.merge( df_ref, df_returns, how='left', on='customer_id' )
df_ref.loc[df_ref['qnt_returns'].isna(), 'qnt_returns'] = 0
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qnt_products     91
frequency        91
qnt_returns       0
dtype: int64

# 4.0 Exploration Data Analysis

In [20]:
df4 = df_ref.dropna().copy()

## 4.3. Estudo do Espaço

In [21]:
# Selected Features
df43 = df4.drop( columns='customer_id', axis=1 )

In [22]:
mm = pp.MinMaxScaler()

df43['gross_revenue']          = mm.fit_transform( df43[['gross_revenue']] )
df43['recency_days']           = mm.fit_transform( df43[['recency_days']] )
df43['qnt_products']           = mm.fit_transform( df43[['qnt_products']] )
df43['frequency']              = mm.fit_transform( df43[['frequency']] )
df43['qnt_returns']            = mm.fit_transform( df43[['qnt_returns']] )

X = df43.copy()

### UMAP

In [23]:
reducer = umap.UMAP( random_state=42 )
embedding = reducer.fit_transform( X )

# Embedding
df_umap = pd.DataFrame()
df_umap['embedding_x'] = embedding[:, 0]
df_umap['embedding_y'] = embedding[:, 1]


# 7.0. Hyper Parameter Fine Tuning

In [24]:
X = df_umap.copy()

# 8.0 Model Training

## 8.1. GMM

In [25]:
k = 12

# Model Definition
gmm_model = mx.GaussianMixture( n_components=k, n_init=300, random_state=42 )

# Model Training
gmm_model.fit( X )

# Model Prediction
labels = gmm_model.predict( X )

## Cluster Validation

In [26]:
# SS (Silhouette Score)
print('SS Value: {}'.format(m.silhouette_score( X, labels, metric='euclidean' ) ) )

SS Value: 0.48913970589637756


# 9.0 Cluster Analyses

In [27]:
df9 = X.copy()
df9['clusters'] = labels

## 9.4. Cluster Profile

In [28]:
df92 = df4.copy()
df92['clusters'] = labels


df92['recency_days'] = df92['recency_days'].astype( np.int64 )
df92['qnt_products'] = df92['qnt_products'].astype( np.int64 )
df92['qnt_returns'] = df92['qnt_returns'].astype( np.int64 )

In [29]:
# Numbers of Customers
df_cluster = df92[['clusters', 'customer_id']].groupby('clusters').count().reset_index().rename(columns={'customer_id': 'qnt_customers'})
df_cluster['perc_customer'] = 100 * ( df_cluster['qnt_customers'] / df_cluster['qnt_customers'].sum() )

# Avg Gross Revenue
df_avg_gross_revenue = df92[['clusters', 'gross_revenue']].groupby( 'clusters' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_gross_revenue, on='clusters', how='inner' )

# Avg Recency Days
df_avg_recency_days = df92[['clusters', 'recency_days']].groupby( 'clusters' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_recency_days, on='clusters', how='inner' )

# Avg Invoice_no
df_avg_products = df92[['clusters', 'qnt_products']].groupby( 'clusters' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_products, on='clusters', how='inner' )

# Frequecy
df_frequency = df92[['clusters', 'frequency']].groupby('clusters').mean().reset_index()
df_cluster = pd.merge( df_cluster, df_frequency, on='clusters', how='inner' )

# qnt_returns
df_returns = df92[['clusters', 'qnt_returns']].groupby('clusters').mean().reset_index()
df_cluster = pd.merge( df_cluster, df_returns, on='clusters', how='inner' )

df_cluster

,clusters,qnt_customers,perc_customer,gross_revenue,recency_days,qnt_products,frequency,qnt_returns
0,0,542,9.500438,553.885018,189.020295,39.391144,1.022140,1.916974
1,1,865,15.162138,1680.522486,27.142197,95.512139,0.032237,15.810405
2,2,530,9.290096,1043.876094,112.741509,73.756604,0.567410,3.262264
3,3,183,3.207713,457.049126,37.344262,29.087432,1.002732,1.371585
4,4,881,15.442594,5959.548820,6.360953,240.113507,0.049229,70.936436
5,5,277,4.855390,906.624801,361.902527,65.126354,1.051507,2.472924
6,6,391,6.853637,520.468721,253.787724,40.276215,1.051151,1.488491
7,7,524,9.184926,941.134427,169.811069,51.727099,0.257886,30.019084
8,8,227,3.978966,2099.911762,48.207048,164.167401,0.187853,10.132159
9,9,581,10.184049,907.195491,63.234079,56.981067,0.777017,7.449225


# 11.0 Deploy to Production

In [30]:
# df92.dtypes

In [31]:
# Database Connection
# conn = create_engine( 'sqlite:///insiders_db.sqlite' )

# Drop Table
#query_drop_insiders = """
#    DROP TABLE insiders
#"""

# Crate table
#query_create_table = """
#    CREATE TABLE insiders(
#        customer_id   INTEGER,
#        gross_revenue REAL,
#        recency_days  INTEGER,
#        qnt_products  INTEGER,
#        frequency     REAL,
#        qnt_returns   INTEGER,
#        clusters      INTEGER
#    )
#"""

#conn.execute( query_drop_insiders )

In [32]:
# insert data into
#df92.to_sql( 'insiders', con=conn, if_exists='append', index=False )#

In [33]:
# Get query
#query_collect = """
#    SELECT * FROM insiders
#"""

#df = pd.read_sql_query( query_collect, conn )

In [34]:
#df.shape